In [1]:
from z3 import *

x = Real('x')
y = Real('y')
s = Solver()
s.add(x + y > 5, x > 1, y > 1)
print(s.check())
print(s.model())

sat
[y = 4, x = 2]


In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)

housing_data = pd.read_csv('./Data/kc_house_data.csv')

In [3]:
sqft_cols = [i for i in housing_data.columns if 'sqft' in i]

In [4]:
X = housing_data[sqft_cols]
y = housing_data['price']
X.head()

,sqft_living,sqft_lot,sqft_above,sqft_basement,sqft_living15,sqft_lot15
0,1180,5650,1180,0,1340,5650
1,2570,7242,2170,400,1690,7639
2,770,10000,770,0,2720,8062
3,1960,5000,1050,910,1360,5000
4,1680,8080,1680,0,1800,7503


In [5]:
from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.experimental import enable_hist_gradient_boosting
# from sklearn.ensemble import HistGradientBoostingRegressor

gbr = GradientBoostingRegressor(
    max_depth=3,
    learning_rate=0.1,
    n_estimators=50  # number of trees
)

gbr.fit(X,y)


GradientBoostingRegressor(n_estimators=50)

In [6]:
import sklearn
help(sklearn.tree._tree.Tree)

Help on class Tree in module sklearn.tree._tree:

class Tree(builtins.object)
 |  Array-based representation of a binary decision tree.
 |  
 |  The binary tree is represented as a number of parallel arrays. The i-th
 |  element of each array holds information about the node `i`. Node 0 is the
 |  tree's root. You can find a detailed description of all arrays in
 |  `_tree.pxd`. NOTE: Some of the arrays only apply to either leaves or split
 |  nodes, resp. In this case the values of nodes of the other type are
 |  arbitrary!
 |  
 |  Attributes
 |  ----------
 |  node_count : int
 |      The number of nodes (internal nodes + leaves) in the tree.
 |  
 |  capacity : int
 |      The current capacity (i.e., size) of the arrays, which is at least as
 |      great as `node_count`.
 |  
 |  max_depth : int
 |      The depth of the tree, i.e. the maximum depth of its leaves.
 |  
 |  children_left : array of int, shape [node_count]
 |      children_left[i] holds the node id of the left child 

In [7]:
gbr.estimators_[0]

array([DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                      random_state=RandomState(MT19937) at 0x1EDB6362140)],
      dtype=object)

In [8]:
# do this just for regression to start

##### HOW TO GENERATE LOGICAL REPRESENTATIONS

### Leaf encoding
# run a DFS on the tree, store nodes in a stack

# when you reach a leaf, write the logical statements for 
# pi(l) which is stack[1:] n wl where wl is the weight value of the leaf

# pruning: using an input x_i', compare the difference at every node in the DFS x_i
# if |x_i-x_i'| < epsilon where epsilon is the predefined robustness factor
# then keep node, otherwise pop current node from the stack and 
# stop searching this branch of the tree (return to previous level)

### Tree encoding
# then represent PI(D) as V(pi(l)) a disjunction of all pi(l) found during DFS

### GBM encoding
# lastly encode the full model instelf, (n(PI(D_i))) n (out=sum(wl_i))
# I haven't fulling figured out how to represent the out=sum(wl_i) part yet.
# wl_i should be the one value returned from each tree PI(D_i), we might just 
# need the additional predicate D_i(x)=wl_i meaning that putting x into the decision tree returns wl_i
# but I'm not totally sure how this is expressed for Z3

### Robust property encoding
# see section 5 of the paper this is easier than the previous stuff but it 
# again has stuff like R(x) that I need to read the documentation on how to define

tree = gbr.estimators_[0,0].tree_



In [9]:
tree